In [1]:
import os
from os import path
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import io
import scipy as scipy
import collections
import scipy.sparse as sp_sparse
import tables
import time
import numba as nb
from numba import config
from numba_progress import ProgressBar
import ipdb

from ABCA7lof2.aggregation import aggregate_fastqs

config.THREADING_LAYER = 'workqueue'



In [2]:
# define in and output paths
path_to_outputs = '/home/gridsan/djuna/homer/github/ABCA7lof2/raw_data/cellranger_counts_out/aggr_out/'

if not path.exists(path_to_outputs):
    os.mkdir(path_to_outputs)
    
meta_path_individual = '/home/gridsan/djuna/homer/github/ABCA7lof2/raw_data/metadata/single_cell_individual_metadata.csv'
counts_path_individual = '/home/gridsan/djuna/homer/github/ABCA7lof2/raw_data/cellranger_counts_out/'

aggregate_fastqs(path_to_outputs, meta_path_individual, counts_path_individual, Ncells=300000, Ngenes=36601)


preparing...
aggregating D20-7432
aggregating D20-7433
aggregating D20-7434
aggregating D20-7435
aggregating D20-7436
aggregating D20-7437
aggregating D20-7438
aggregating D20-7439
aggregating D20-7440
aggregating D20-7441
aggregating D20-7442
aggregating D20-7443
aggregating D20-7444
aggregating D20-7445
aggregating D20-7446
aggregating D20-7447
aggregating D20-7448
aggregating D20-7450
aggregating D20-7451
aggregating D20-7452
aggregating D20-7453
aggregating D20-7454
aggregating D20-7455
aggregating D20-7456
aggregating D20-7457
aggregating D20-7458
aggregating D17-8792
aggregating D17-8782
aggregating D17-8796
aggregating D17-8764
aggregating D17-8776
aggregating D17-8772
aggregating D17-8768
aggregating D17-8766
aggregating D17-8758
aggregating D17-8778
aggregating D17-8756
aggregating D17-8800
aggregating D17-8790
aggregating D17-8754
aggregating D17-8760
aggregating D17-8786
saving metadata
truncating arrays
done.


In [5]:
p = './raw_data/cellranger_counts_out/aggr_out/counts.npy'

In [14]:
open(p, 'r+').truncate()

0

In [13]:
f

<_io.TextIOWrapper name='./raw_data/cellranger_counts_out/aggr_out/counts.npy' mode='r+' encoding='ISO-8859-1'>

In [9]:
f.truncate(2)

2

In [81]:
np.save(features_path, np.empty(shape=3, dtype='str'))
features_out = np.lib.format.open_memmap(features_path, mode='r+', dtype='str')


In [82]:
features_out

memmap(['', '', ''], dtype='<U1')

In [70]:
np.empty(shape=3, dtype='str')

array(['', '', ''], dtype='<U1')

In [104]:
x = np.empty((2,5), dtype=object)
x[0][0] = 1

In [106]:
x[0][2] = 'string'


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import io
import scipy as scipy
import collections
import scipy.sparse as sp_sparse
import tables
import time
import numba as nb
from numba_progress import ProgressBar
import os


def decode(numpy_array):
    return np.array([numpy_array[x].decode("utf-8") for x in range(len(numpy_array))])

def write_matrix(ID, counts_path_individual, cell_counts, cell_meta, subset_meta, counter, features_id_out, features_name_out, barcodes_out, Ngenes, i):
    #ipdb.set_trace()


    print('loading matrix...')
    matrix = get_matrix_from_h5(counts_path_individual + ID + '/outs/filtered_feature_bc_matrix.h5')
    
    mat_array = matrix.matrix.T.toarray()
    temp_ncells = mat_array.shape[0]
    
    barcodes_out[counter:counter+temp_ncells]  = decode(np.array(matrix.barcodes))
    start = i*Ngenes
    stop = start+Ngenes
    features_id_out[start:stop] = decode(np.array(matrix.feature_ref['id']))
    features_name_out[start:stop] = decode(np.array(matrix.feature_ref['name']))
    
    print('writing to memmap...')
    write_counts_to_memmap(mat_array, cell_counts, counter)
    add_metadata(cell_meta, subset_meta, counter, temp_ncells)
    
    counter+=temp_ncells
    
    return counter

@nb.njit(parallel=True)
def write_counts_to_memmap(mat_array, cell_counts, counter):
    for x in nb.prange(mat_array.shape[0]):
        cell_counts[counter+x] = mat_array[x]

def add_metadata(cell_meta, subset_meta, counter, temp_ncells):
    for x in range(temp_ncells):
        cell_meta[counter+x] = subset_meta
        
def get_metadata(meta, Library_ID, barcodes):
    print('getting metadata...')
    subset_meta = np.array(meta[meta['sample_id']==Library_ID])[0]
    subset_meta = np.array([subset_meta,]*len(barcodes))
    subset_meta = np.concatenate((barcodes[:,None], subset_meta), axis = 1)
    
    return subset_meta

def aggregate_fastqs(path_to_outputs, meta_path_individual, counts_path_individual, Ncells=300000, Ngenes=36601):
    print('preparing...')
    # initiate empty counts matrix and load as memmap
    counts_path = path_to_outputs + 'counts.npy'
    np.save(counts_path, np.empty(shape=(Ncells, Ngenes), dtype='int32'))
    cell_counts = np.lib.format.open_memmap(counts_path, mode='r+', dtype='int32', shape=(Ncells, Ngenes))

    # load metadata and get sample IDs
    ind_meta = pd.read_csv(meta_path_individual)
    n_metavars = ind_meta.shape[1]
    meta_path = path_to_outputs + 'metadata.npy'
    cell_meta = np.empty(shape=(Ncells, n_metavars), dtype=object)
    
    # initiate barcodes and features
    sample_ids = ind_meta['sample_id']
    n_ind = len(sample_ids)
    features_id_path = path_to_outputs + 'features_id.npy'
    np.save(features_id_path, np.empty(shape=Ngenes*n_ind, dtype='str'))
    features_id_out = np.lib.format.open_memmap(features_id_path, mode='r+', dtype='str')

    features_name_path = path_to_outputs + 'features_name.npy'
    np.save(features_name_path, np.empty(shape=Ngenes*n_ind, dtype='str'))
    features_name_out = np.lib.format.open_memmap(features_name_path, mode='r+', dtype='str')

    barcodes_path = path_to_outputs + 'barcodes.npy'
    np.save(barcodes_path, np.empty(shape=Ncells, dtype='str'))
    barcodes_out = np.lib.format.open_memmap(barcodes_path, mode='r+', dtype='str')
 
    # combine matrices
    counter = 0    
    counter = combine_matrices(sample_ids, cell_meta, ind_meta, cell_counts, counts_path_individual, features_id_out, features_name_out, barcodes_out, counter, Ngenes)

    # save metadata
    np.save(meta_path, cell_meta)
    
    # truncate arrays                 
    remove = Ncells-counter
    truncate(counts_path, cell_counts.size - remove * cell_counts[0].size)   
    truncate(meta_path, cell_meta.size - remove * cell_meta[0].size)   
    truncate(barcodes_path, barcodes_out.size - remove * barcodes_out[0].shape)   
                                             
def combine_matrices(sample_ids, cell_meta, ind_meta, cell_counts, counts_path_individual, features_id_out, features_name_out, barcodes_out, counter, Ngenes):
    
    for i in (range(len(sample_ids))):
        ID = sample_ids[i]
        print('aggregating ' + str(ID))
        subset_meta = np.array(ind_meta[ind_meta['sample_id']==ID])[0]
        counter = write_matrix(ID, counts_path_individual, cell_counts, cell_meta, subset_meta, counter, features_id_out, features_name_out, barcodes_out, Ngenes, i)
    return counter



In [6]:
cell_counts

OSError: 5506840056 requested and 0 written

In [5]:
np.max(all_data[:counter])

2557

In [12]:
matrix = get_matrix_from_h5(path_to_counts + 'D20-7432' + '/outs/filtered_feature_bc_matrix.h5')

loading hd5..


In [15]:
matrix.matrix.T.shape

(1840, 36601)

In [ ]:
all_data[:

In [17]:
meta = pd.read_csv(path_to_meta)
all_counts = np.memmap(matrix_path, mode = 'w+', shape = (3000000, 36601), dtype = 'int32')
all_meta = np.empty([0,meta.shape[1]+1])
sample_ids = np.array(meta['sample_id'])
counter = 0
initial_id = sample_ids[0]
init_features, init_barcodes, counter = write_matrix(initial_id, path_to_counts, all_counts, counter)
init_meta = get_metadata(meta, initial_id, init_barcodes)

writing matrix...
loading hd5..
writing to memmap...
getting metadata...


In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy import io
import scipy as scipy
import collections
import scipy.sparse as sp_sparse
import tables
import time
import numba as nb
from numba_progress import ProgressBar
import os

#TODO: add docstrings to each function
def get_matrix_from_h5(filename):
    print('loading hd5..')
    #TODO: understand this function better - describe all the slots in the doc string
    '''
    function adapted from https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/advanced/h5_matrices
    Args:
        filename (str) : path pointing to the counts matrix (genes x cell) output by the cellranger count function
        This file contains the counts for a single sample in a slot called 'matrix'
        Corresponding cell barcodes (column-names) are in the slot called 'barcodes' (must be unique)
    Returns:
        #TODO: specify what is being returned
    '''
    CountMatrix = collections.namedtuple('CountMatrix', ['feature_ref', 'barcodes', 'matrix'])
    with tables.open_file(filename, 'r') as f:
        mat_group = f.get_node(f.root, 'matrix')
        barcodes = f.get_node(mat_group, 'barcodes').read()
        data = getattr(mat_group, 'data').read()
        indices = getattr(mat_group, 'indices').read()
        indptr = getattr(mat_group, 'indptr').read()
        shape = getattr(mat_group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
        feature_ref = {}
        feature_group = f.get_node(mat_group, 'features')
        feature_ids = getattr(feature_group, 'id').read()
        feature_names = getattr(feature_group, 'name').read()
        feature_types = getattr(feature_group, 'feature_type').read()
        feature_ref['id'] = feature_ids
        feature_ref['name'] = feature_names
        feature_ref['feature_type'] = feature_types
        tag_keys = getattr(feature_group, '_all_tag_keys').read()
        for key in tag_keys:
            key = key.decode("utf-8")
            feature_ref[key] = getattr(feature_group, key).read()
        return CountMatrix(feature_ref, barcodes, matrix)

def decode(numpy_array):
    return np.array([numpy_array[x].decode("utf-8") for x in range(len(numpy_array))])

def write_matrix(Library_ID, path_to_counts, all_counts, counter):
    print('writing matrix...')
    #ipdb.set_trace()
    matrix = get_matrix_from_h5(path_to_counts + Library_ID + '/outs/filtered_feature_bc_matrix.h5')
    barcodes = decode(np.array(matrix.barcodes))
    features = np.concatenate((decode(np.array(matrix.feature_ref['id']))[:,None],decode(np.array(matrix.feature_ref['name']))[:,None]), axis = 1)
    mat = matrix.matrix
    mat_array = mat.toarray()
    #start = time.time()
    print('writing to memmap...')
    #with ProgressBar(total=mat_array.shape[0]) as numba_progress:
    write_to_memmap(mat_array, all_counts, counter)
    #end = time.time()
    #print(end - start)
    counter+=mat_array.shape[0]
    return features, barcodes, counter

@nb.njit(parallel=True)
def write_to_memmap(mat_array, all_counts, counter):
    '''
    writes array object to memmap
    Args:
        mat_array:
        all_counts:
        counter:
        progress_hook:
    '''
    for i in nb.prange(mat_array.shape[0]):
        all_counts[counter+i] = mat_array[i]
        #progress_hook.update(1)

def get_metadata(meta, Library_ID, barcodes):
    print('getting metadata...')
    subset_meta = np.array(meta[meta['sample_id']==Library_ID])[0]
    subset_meta = np.array([subset_meta,]*len(barcodes))
    subset_meta = np.concatenate((barcodes[:,None], subset_meta), axis = 1)
    return subset_meta

def aggregate_fastqs(path_to_meta, path_to_counts, features_path, meta_path, matrix_path):
    meta = pd.read_csv(path_to_meta)
    all_counts = np.memmap(matrix_path, mode = 'w+', shape = (3000000, 36601), dtype = 'int32')
    all_meta = np.empty([0,meta.shape[1]+1])
    sample_ids = np.array(meta['sample_id'])

    counter = 0
    initial_id = sample_ids[0]
    print('aggregating ' + str(initial_id))
    init_features, init_barcodes, counter = write_matrix(initial_id, path_to_counts, all_counts, counter)
    init_meta = get_metadata(meta, initial_id, init_barcodes)
    all_meta = np.concatenate((all_meta, init_meta), axis = 0)
    np.save(features_path,init_features)
    print('done.')
    
    return all_counts
    #combine_matrices(sample_ids, init_features, all_meta, meta, all_counts, counter, path_to_counts, meta_path, matrix_path)

def combine_matrices(sample_ids, init_features, all_meta, meta, all_counts, counter, path_to_counts, meta_path, matrix_path):
    for i in (range(len(sample_ids)-1)):
        #print(sample_ids[i+1])
        print('aggregating ' + str(sample_ids[i+1]))
        features, barcodes, counter = write_matrix(sample_ids[i+1], path_to_counts, all_counts, counter)
        meta_temp = get_metadata(meta, sample_ids[i+1], barcodes)
        all_meta = np.concatenate((all_meta, meta_temp), axis = 0)
        print('done')
        if not np.array_equal(features, init_features):
            raise ValueError('feature matrices are inconsistent for ' + sample_ids[i+1])
            break
    print('saving data')
    all_counts[:counter].tofile(matrix_path)
    np.save(meta_path, all_meta)
    print('all done.')


In [7]:
@nb.njit(parallel=True)
def write_to_memmap(x):
    for i in nb.prange(x):
        1+1

In [4]:
meta = pd.read_csv(path_to_meta)

0      84.435318
1      85.533196
2      85.694730
3      79.687885
4      74.050650
5      89.393566
6     100.766598
7      89.360712
8      93.801506
9      91.715264
10     85.114305
11     83.485284
12    100.008214
13     86.548939
14     90.759754
15     94.343600
16     79.408624
17     89.448323
18     94.004107
19     86.505133
20     89.418207
21     82.255989
22     92.624230
23     77.459274
24     88.950034
25     94.598220
26     87.238877
27     80.533881
28     93.237509
29     76.057495
30     82.710472
31     89.713895
32     83.728953
33     85.776865
34     74.773443
35     92.479124
36     88.468172
37     85.828884
38     87.665982
39     89.026694
40     86.403833
41     87.449692
Name: age_death, dtype: float64

In [ ]:
# load the data
echo(f"loading the data", underline=True)
meta, features, counts = get_data(args.meta_path, args.features_path, args.matrix_path)
gene_names = features[:,-1].astype('U17').reshape(-1)
meta_col_names = np.load(args.meta_col_names, allow_pickle=True)

 # filter by cells prior to annotation
echo(f"initial filtering of cells", underline=True)
pdf = matplotlib.backends.backend_pdf.PdfPages(path_to_outputs+"/output.pdf")
mito_index, total_counts, mito_fractions, filtered_meta, filtered_counts = filter_cells(gene_names, counts, meta, args.sample_size, args.lower_bound_counts, args.upper_bound_counts, pdf, args.mito_prefix, path_to_outputs)

# annotate the major cell types
echo(f"annotating major cell types", underline=True)
annotations, marker_out, projected_matrix, predict, scores, marker_genes, marker_indices = get_major_annotations(args.marker_path, gene_names, filtered_counts, total_counts, args.sample_size, args.n_components_pca, args.n_components_gaussian, path_to_outputs, args.infer_N_markers)

# filter by Gaussian logliklihood
echo(f"filtering by logliklihood", underline=True)
keep_cells_1 = filter_on_gaussian_logliklihood(scores)

# filter by major cell type annotation
echo(f"intra-celltype filtering", underline=True)
individual_annotation = filtered_meta[:,1]
keep_cells_2, keep_cells_mito, keep_cells_individual, keep_genes, keep_individuals = filter_cells_by_major_annotation(mito_fractions, total_counts, args.sample_size, predict, individual_annotation, filtered_counts)

# compile everything into the final metadata
echo(f"compiling all the data", underline=True)
filtered_meta_final = np.concatenate((filtered_meta,(keep_cells_1&keep_cells_2).reshape(-1,1), keep_cells_mito.reshape(-1,1), keep_cells_individual.reshape(-1,1), predict.reshape(-1,1), scores.reshape(-1,1), annotations.reshape(-1,1), total_counts.reshape(-1,1), mito_fractions.reshape(-1,1)), axis = 1)
meta_names = np.concatenate((meta_col_names, np.array(('keep_cells_all_per_celltype', 'keep_cells_mito_per_celltype', 'keep_cells_individual', 'predictions', 'gaussian_liklihood', 'annotations', 'total_counts', 'mito_fractions'))), axis = 0)

# save
echo(f"saving all the data", underline=True)
save_all(path_to_outputs, filtered_meta_final, meta_names, keep_genes, gene_names, marker_indices, mito_index, projected_matrix, annotations)
